Tesing on binary classification problems and have data with nominal-valued attributes and no missing
values (weather.nominal, titanic, vote.noUnknowns

In [2]:
import pandas as pd
import numpy as np

In [3]:
def readArff(filename):
    with open ('./NominalData/'+filename+'.arff', 'r') as f:
        # split lines, remove ones with comments
        lines = [line.lower() for line in f.read().split('\n') if not line.startswith('%')]
        
    # remove empty lines
    lines = [line for line in lines if line != '']
    
    columns = []
    data = []
    for index, line in enumerate(lines):
        if line.startswith('@attribute'):
            columns.append(line)
            
        if line.startswith('@data'):
            # get the rest of the lines excluding the one that says @data
            data = lines[index+1:]
            break
            
    # clean column names -- '@attribute colname  \t\t\t{a, b, ...}'
    cleaned_columns = [c[11:c.index('{')].strip() for c in columns]
    
    # clean and split data
    cleaned_data = [d.replace(', ', ',').split(',') for d in data]
    
    # create dataframe
    return pd.DataFrame(cleaned_data, columns = cleaned_columns)

In [4]:
def preprocess_data(df):
    # change class values to {-1, 1}
    y, unique = pd.factorize(df.iloc[:,-1])
    new_y = np.where(y==0, -1, 1)
    assert set(new_y) == {-1, 1}, 'Response variable must be ±1'
    
    # change xs to 2d numpy array
    xs = df.iloc[:,:-1]
    xs = xs.values
    
    return xs, new_y

In [5]:
def accuracy_score(y_true, y_pred):
    """ Compare y_true to y_pred and return the accuracy """
    accuracy = np.sum(y_true == y_pred, axis=0) / len(y_true)
    return accuracy

In [6]:
class DecisionStump:
    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.n_features = np.shape(self.X)[1]
        self.info_gain = None
        self.error = None
        self.best_attribute = None
        self.tree = dict()
        self.predictions = None
        self.class_val = 1

    def __str__(self):
        return f"""information_gain: {self.info_gain}, error: {self.error}, feature:{self.best_attribute}"""
    
    
    def _entropy(self, col):
        """
        Calculate the entropy with respect to the target column.
        """
        vals, counts = np.unique(col, return_counts = True)

        entropy = np.sum([(-counts[i]/np.sum(counts)) * np.log2(counts[i]/np.sum(counts)) 
                          for i in range(len(vals))])
        return entropy
    
    
    def _information_gain(self, attr): 
        # calculate the entropy of the total dataset
        total_entropy = self._entropy(self.y)

        # calculate the sum of the weighted entropy of the attributes
        vals, counts = np.unique(attr, return_counts=True)


        weighted_entropy = sum([(counts[i]/np.sum(counts)) * 
                            self._entropy(self.y[(attr == vals[i])]) for i in range(len(vals))])

        # calculate information gain
        info_gain = total_entropy - weighted_entropy
        return info_gain
    
    def _make_tree(self):
        # predict values based on self.best_attribute
        attr = self.X[:, self.best_attribute]
        vals, counts = np.unique(attr, return_counts=True)
        
        # tree = {attr_val1: p(1), attr_val2, p(1)}
        # keys represent branches, values represent probability of 1
        # we know the y's are {-1, 1}
        for val in vals:
            subset = self.y[(attr == val)]
            new_subset = np.where(subset == -1, 0, 1) # replace -1 with 0
            prob = sum(new_subset) / len(new_subset)
            self.tree[val] = prob
            
    def _predict(self): 
        # predict values based on self.best_attribute
        attr = self.X[:, self.best_attribute]
        self.predictions = np.ones(np.shape(self.y))

        for i, x_i in enumerate(attr):
            if self.tree[x_i] < 0.5:
                self.predictions[i] = -1
            
            if self.tree[x_i] == 0.5:
                # pick more probable class from overall class
                self.predictions[i] = np.sign(np.sum(self.y))
    
    
    def _calculate_error(self):        
        self._make_tree()
        self._predict()
        
        # calculate percent inaccuracy
#         assert np.shape(self.predictions) == np.shape(self.y) # sanity check
#         accuracy = np.sum(self.predictions == self.y, axis=0) / len(self.y)
#         self.error = 1 - accuracy
    
    def learn(self):
        max_gain = float('-inf')

        for f in range(self.n_features):
            gain = self._information_gain(self.X[:, f])
            
            if max_gain < gain:
                self.info_gain = gain
                self.best_attribute = f
                max_gain = gain
        self._calculate_error()

In [7]:
class AdaBoost:
    
    def __init__(self, T=20):
        self.T = T
    
    def _resample(self, X, y, w):
        # sanity checks 
        assert sum(w) > 0.999999
        assert sum(w) < 1.000001

        assert np.shape(w)[0] == X.shape[0]

        # combine into dataframe
        xs = pd.DataFrame(X)
        df = pd.concat([xs, pd.DataFrame(y, columns=["y"])], axis=1)

        # resample
        new_data = df.sample(frac=1, replace=True, weights=w, random_state=1)
        new_xs = new_data.iloc[:,:-1].values
        new_ys = new_data.iloc[:, -1].values
        return new_xs, new_ys
    
    def train(self, X, y):
        epsilon = 1e-6 # add stability -- avoid div_by_0 error for when learner.error = 0

        n_instances = np.shape(X)[0]
        self.stumps = np.zeros(shape=self.T, dtype=object)
        self.alphas = np.zeros(shape=self.T)

        # initialize weights uniformly
        weights = np.ones(shape=n_instances) / n_instances
        # automatically use the whole data for first sample
        # since all weights are even
#         sample_X = X
#         sample_y = y

        for t in range(self.T):
            # use updated weights to resample
            sample_X, sample_y = self._resample(X, y, weights)
            
            learner = DecisionStump(sample_X, sample_y)
            learner.learn()
#             print(f"Iteration: {t}, Missed: {np.sum(np.where(sample_y != learner.predictions, 1, 0))}")

            error = sum(weights[sample_y != learner.predictions]) / sum(weights)
            if error > 0.5:
                error = 1 - error
                learner.class_val = -1
        
            alpha = 0.5 * np.log((1.0 - error + epsilon) / (error + epsilon))

            weights *= np.exp(-alpha * sample_y * learner.predictions * learner.class_val)
            weights /= weights.sum()
            
#             sample_X, sample_y = self._resample(X, y, weights)
            
            # save stump objects and alphas
            self.stumps[t] = learner
            self.alphas[t] = alpha
            
    
    
    def predict(self, X):
        n_samples = np.shape(X)[0]
        y_pred = np.zeros((n_samples, 1))

        # For each classifier => label the samples
        for stump, alpha in zip(self.stumps, self.alphas):
            attr = X[:, stump.best_attribute]
            predictions = np.ones(np.shape(y_pred)) # Set all predictions to '1' initially
            
            for i, x_i in enumerate(attr):
                val = stump.tree.get(x_i, None) # guard against lookup not in tree for attribute

                if val is None or val < 0.5:
#                 if stump.tree[x_i] < 0.5:
                    predictions[i] = -1 # switch predictions to -1 if p(1) < 0.5

            # Add predictions weighted by the classifiers alpha (alpha indicative of classifier's proficiency)
            y_pred += alpha * predictions * stump.class_val

        # Return sign of prediction sum
        y_pred = np.sign(y_pred).flatten()

        return y_pred

In [7]:
X,y = preprocess_data(readArff('weather.nominal'))
ada = AdaBoost()
ada.train(X,y)
preds = ada.predict(X)
accuracy_score(preds, y)

0.7142857142857143

In [63]:
X,y = preprocess_data(readArff('titanic'))
ada = AdaBoost()
ada.train(X,y)
preds = ada.predict(X)
accuracy_score(preds, y)

0.7794943820224719

In [64]:
X,y = preprocess_data(readArff('vote.noUnknowns'))
ada = AdaBoost()
ada.train(X,y)
preds = ada.predict(X)
accuracy_score(preds, y)

0.9698275862068966

In [8]:
def k_fold_cross_val(filename, T, k):
    X, y = preprocess_data(readArff(filename))

    # Group examples s.t. fold sizes differ by at most 1.
    # Assign [N = floor(len(data) / k)] examples to each fold.
    # Assign one additional example to [r = len(data) % k] folds.
    groups_X = []
    groups_y = []
    size_per_group = len(X) // k
    r = len(X) % k
    start = 0
    for i in range(k):
        n_examples = size_per_group + (i < r)
        groups_X.append(X[start: start + n_examples])
        groups_y.append(y[start: start + n_examples])
        start += n_examples

    # cross validation each fold
    total_correct = total_data = 0
    for i in range(k):
        print("\nUsing group {} of {} as test data".format(i+1, k))
        train_data_X = np.array([x for group in groups_X[:i] + groups_X[i+1:] for x in group])
        train_data_y = np.array([x for group in groups_y[:i] + groups_y[i+1:] for x in group])
        test_data_X = np.array(groups_X[i])
        test_data_y = np.array(groups_y[i])

        ada = AdaBoost(T)
        ada.train(train_data_X, train_data_y)
        preds = ada.predict(test_data_X)
        n_correct = np.sum(preds == test_data_y, axis=0)
        n_data = len(preds)
        total_correct += n_correct
        total_data += n_data

    avg_acc = 100 * total_correct / total_data
    print("\nAverage accuracy: {:.2f}% ({}/{})"
          .format(avg_acc, total_correct, total_data))

In [9]:
k_fold_cross_val('weather.nominal', 25, 14)


Using group 1 of 14 as test data

Using group 2 of 14 as test data

Using group 3 of 14 as test data

Using group 4 of 14 as test data

Using group 5 of 14 as test data

Using group 6 of 14 as test data

Using group 7 of 14 as test data

Using group 8 of 14 as test data

Using group 9 of 14 as test data

Using group 10 of 14 as test data

Using group 11 of 14 as test data

Using group 12 of 14 as test data

Using group 13 of 14 as test data

Using group 14 of 14 as test data

Average accuracy: 64.29% (9/14)


In [10]:
k_fold_cross_val('titanic', 25, 10)


Using group 1 of 10 as test data

Using group 2 of 10 as test data

Using group 3 of 10 as test data

Using group 4 of 10 as test data

Using group 5 of 10 as test data

Using group 6 of 10 as test data

Using group 7 of 10 as test data

Using group 8 of 10 as test data

Using group 9 of 10 as test data

Using group 10 of 10 as test data

Average accuracy: 77.95% (555/712)


In [11]:
k_fold_cross_val('vote.noUnknowns', 20, 10)


Using group 1 of 10 as test data

Using group 2 of 10 as test data

Using group 3 of 10 as test data

Using group 4 of 10 as test data

Using group 5 of 10 as test data

Using group 6 of 10 as test data

Using group 7 of 10 as test data

Using group 8 of 10 as test data

Using group 9 of 10 as test data

Using group 10 of 10 as test data

Average accuracy: 96.98% (225/232)
